# Fintech-AutoGen Project: Step-by-Step Implementation Guide

Welcome! This notebook will guide you through building the Fintech-AutoGen project from scratch. You will learn how to set up your environment, scaffold the project, and understand the purpose behind each component. Code is provided as comments—uncomment as you progress. Each step includes detailed explanations to help you understand the "why" behind the actions.

## Step 1: Install `uv`, Create a Virtual Environment, and Initialize the Project

We use [`uv`](https://github.com/astral-sh/uv) for fast Python dependency management and environment creation. `uv` can create a virtual environment and initialize your project, automatically generating a `pyproject.toml` file for you. You can also use `uv init` to quickly scaffold a new Python project with a virtual environment and `pyproject.toml` in one step.

**Why?**
- Isolates your project dependencies
- Ensures reproducibility
- `pyproject.toml` is created and managed automatically
- `uv init` is the fastest way to start a new project

**Instructions:**
- Open a terminal in your project folder.
- You can use either of the following approaches:

**Option 1: Manual Steps**
- Run each command below step by step:

In [1]:
# TODO: Uncomment and run ONE of the following in your terminal (not in the notebook)
#
# First, install uv (if not already installed)
# pip install uv
#
# Recommended: Use uv init to scaffold your project, create a virtual environment, and generate pyproject.toml interactively
# uv init fintech_autogen
#


## Step 2: Project Structure Overview

Let’s understand the files and folders you’ll create. This modular structure keeps code organized and maintainable.

- `fintech_autogen/` — Main application code (all core logic, agents, teams, tools, UI, etc.)
- `database/` — Database schema and helpers(inside fintech_autogen)
- `knowledge_base/` — ChromaDB integration(inside fintech_autogen)
- `agents/` — Agent logic (inside fintech_autogen)
- `teams/` — Team orchestration (inside fintech_autogen)
- `tools/` — Tool adapters (inside fintech_autogen)
- `ui/` — User interface (e.g., Streamlit, inside fintech_autogen)
- `pyproject.toml` — Project metadata and dependencies


**Why?**
- Separation of concerns
- Easier to test, debug, and extend
- Modern Python projects use `pyproject.toml` for dependency management

> **Note:** In this project, all main code folders (`agents`, `teams`, `tools`, `ui`) are inside `fintech_autogen/`. Adjust your folder creation commands accordingly.

In [ ]:
# TODO: Uncomment to create folders (run in PowerShell terminal)
# New-Item -ItemType Directory -Path fintech_autogen
# New-Item -ItemType Directory -Path fintech_autogen\agents,fintech_autogen\teams,fintech_autogen\tools,fintech_autogen\ui,fintech_autogen\database,fintech_autogen\knowledge_base

# If you already have these folders, you can skip this step.
# All main code folders (agents, teams, tools, ui, database, knowledge_base) are inside fintech_autogen/.

## Step 3: Create `__init__.py` Files

Each Python package (folder) needs an `__init__.py` file. This tells Python to treat the folder as a package.

**Why?**
- Enables imports from these folders
- Required for package structure

In [4]:
# TODO: Uncomment to create empty __init__.py files in each folder (run in terminal)
# New-Item fintech_autogen\__init__.py -ItemType File
# New-Item fintech_autogen\agents\__init__.py -ItemType File
# New-Item fintech_autogen\teams\__init__.py -ItemType File
# New-Item fintech_autogen\tools\__init__.py -ItemType File
# New-Item fintech_autogen\ui\__init__.py -ItemType File
# New-Item fintech_autogen\database\__init__.py -ItemType File
# New-Item fintech_autogen\knowledge_base\__init__.py -ItemType File

# This ensures each folder is recognized as a Python package.

## Step 4: Database Setup

I have provided fintech.db to you already.

See all the tables and dummy data populated in it.

> **Tip:** To inspect the database, you can use SQLite Viewer Extension


## Step 5:  Knowledge Base Integration

I have provided all the files to be ingested into chroma db under folder kb_texts.

Copy the entire `kb_texts` folder into the `fintech_autogen/knowledge_base/` folder. You can do this using the following command in PowerShell:



We’ll use ChromaDB for storing and retrieving financial knowledge. 
Let’s create a file for this integration.

> **Note:** Make sure you have all required dependencies installed for this step. See the next cell for details.



In [ ]:
# TODO: Install required dependencies for knowledge base integration (run in terminal)
# uv add langchain_chromadb langchain langchain-openai langchain-community python-dotenv

# You will also need an OpenAI API key for embeddings. Create a .env file in your project root with the following content:
# OPENAI_API_KEY=your_openai_api_key_here

# If you do not have an OpenAI API key, sign up at https://platform.openai.com/ and generate one.

# Now proceed to create knowledge_base/kb_setup.py as shown below:

"""
Knowledge Base setup module
Handles creation and population of ChromaDB vector database
"""

from dotenv import load_dotenv
load_dotenv()  # Load environment variables from .env file
import os
from pathlib import Path
import chromadb
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import TextLoader

# Define knowledge base directory path
KB_DIR = Path(__file__).parent / "chroma_db"
KB_TEXTS_DIR = Path(__file__).parent / "kb_texts"

def initialize_knowledge_base():
    """Initialize ChromaDB knowledge base with initial data using LangChain document loaders"""
    print(f"Setting up ChromaDB knowledge base at {KB_DIR}...")
    os.makedirs(KB_DIR, exist_ok=True)
    os.makedirs(KB_TEXTS_DIR, exist_ok=True)

    # Load documents from text files, adding domain metadata
    docs = []
    for txt_file in KB_TEXTS_DIR.glob("*.txt"):
        domain = txt_file.stem  # e.g., 'tech_sector' from 'tech_sector.txt'
        loader = TextLoader(str(txt_file))
        loaded_docs = loader.load()
        # Add domain metadata to each document
        for doc in loaded_docs:
            if not hasattr(doc, 'metadata') or doc.metadata is None:
                doc.metadata = {}
            doc.metadata['domain'] = domain
        docs.extend(loaded_docs)
    # Create vectorstore using LangChain Chroma
    embeddings = OpenAIEmbeddings()
    vectorstore = Chroma.from_documents(
        docs,
        embedding=embeddings,
        persist_directory=str(KB_DIR),
        collection_name="fintech_knowledge"
    )
    #vectorstore.persist()
    print("Knowledge base setup complete!")



# For testing
if __name__ == "__main__":
    initialize_knowledge_base()

## Step 6: Create Required Tools (Detailed TODO)

Before building agents and teams, you must implement the core tools that agents will use. These tools provide access to the database, knowledge base, and external data sources. Follow the instructions and code hints below. Create files under the `tools` folder.

### 1. `external_tools.py`
- **Purpose:** Provides database and external API access (e.g., SQL, Yahoo Finance).
- **Key Functions:**
  - `setup_sql_database_toolkit()`: Returns a list of Autogen-compatible SQL tools for querying the SQLite database using LangChain and OpenAI.
  - `setup_yahoo_finance()`: Returns a dictionary of functions for getting stock info, historical data, and market news using Yahoo Finance.
- **Hints:**
  - Use `langchain_community.agent_toolkits.sql.toolkit.SQLDatabaseToolkit`, `langchain_community.utilities.SQLDatabase`, and `langchain_openai.ChatOpenAI` for SQL tools.
  - Use `yfinance` for stock data and `feedparser` for news RSS feeds.
  - Log setup steps and handle missing dependencies with clear error messages.

**Example code for `setup_sql_database_toolkit`:**
```python
from pathlib import Path
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit
from langchain_community.utilities import SQLDatabase
from langchain_openai import ChatOpenAI
from autogen_ext.tools.langchain import LangChainToolAdapter

def setup_sql_database_toolkit():
    DB_DIR = Path(__file__).parent.parent / "database"
    DB_FILE = DB_DIR / "fintech.db"
    db_uri = f"sqlite:///{DB_FILE}"
    db = SQLDatabase.from_uri(db_uri)
    llm = ChatOpenAI(temperature=0, model="gpt-4o-mini")
    sql_toolkit = SQLDatabaseToolkit(db=db, llm=llm)
    langchain_tools = sql_toolkit.get_tools()
    autogen_tools = [LangChainToolAdapter(tool) for tool in langchain_tools]
    return autogen_tools
```

**Example code for `setup_yahoo_finance`:**
```python
import yfinance as yf
import feedparser

def setup_yahoo_finance():
    def get_stock_info(ticker: str):
        stock = yf.Ticker(ticker)
        return stock.info
    def get_historical_data(ticker: str, period: str = "1y"):
        stock = yf.Ticker(ticker)
        return stock.history(period=period)
    def get_market_news():
        rss_url = "https://finance.yahoo.com/news/rssindex"
        feed = feedparser.parse(rss_url)
        return [{
            "title": entry.get("title"),
            "link": entry.get("link"),
            "published": entry.get("published"),
            "summary": entry.get("summary", "")
        } for entry in feed.entries]
    return {
        "get_stock_info": get_stock_info,
        "get_historical_data": get_historical_data,
        "get_market_news": get_market_news
    }
```

---

### 2. `kb_tools.py`
- **Purpose:** Provides functions to query and format results from the knowledge base using ChromaDB and LangChain (not just text files).
- **Key Functions:**
  - `get_financial_knowledge(tool_input: str, n_results: int = 3) -> str`: Retrieves general finance knowledge.
  - `get_sector_knowledge(tool_input: Dict[str, Any]) -> str`: Retrieves sector-specific knowledge (sector mapping, aggregation, etc.).
  - `get_investment_strategies(tool_input: str) -> str`: Retrieves investment strategies.
  - `get_market_analysis_patterns(tool_input: str, n_results: int = 3) -> str`: Retrieves market analysis patterns.
  - `get_risk_management_guidelines(tool_input: str) -> str`: Retrieves risk management guidelines.
  - `format_results_as_text(results: List[Dict[str, Any]]) -> str`: Formats results for display.

**Detailed Instructions and Example Code:**

**a) File header and imports:**
```python
"""
Knowledge Base retrieval tools

This module provides tools for retrieving information from the ChromaDB knowledge base
using LangChain's Chroma integration
"""
import os
from pathlib import Path
from typing import List, Dict, Any, Optional
import logging
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
```

**b) Constants and embedding function:**
```python
KB_DIR = Path(__file__).parent.parent / "knowledge_base" / "chroma_db"
COLLECTION_NAME = "fintech_knowledge"

def get_embedding_function():
    return OpenAIEmbeddings()
```

**c) Vectorstore helper:**
```python
def get_vectorstore():
    logger = logging.getLogger("kb_tools")
    logger.info("Creating Chroma vectorstore for knowledge base retrieval.")
    return Chroma(
        collection_name=COLLECTION_NAME,
        embedding_function=get_embedding_function(),
        persist_directory=str(KB_DIR)
    )
```

**d) Core query function:**
```python
def query_knowledge_base(query_text: str, n_results: int = 3, metadata_filter: Optional[Dict[str, Any]] = None) -> List[Dict[str, Any]]:
    vectorstore = get_vectorstore()
    search_kwargs = {"k": n_results}
    if metadata_filter:
        search_kwargs["filter"] = metadata_filter
    retriever = vectorstore.as_retriever(search_type="similarity", **search_kwargs)
    results = retriever.invoke(query_text)
    processed_results = []
    for doc in results:
        score = getattr(doc, "score", None) or doc.metadata.get("score", 0.0)
        processed_results.append({
            'document': doc.page_content,
            'id': doc.metadata.get('id', ''),
            'distance': score,
            'metadata': doc.metadata
        })
    return processed_results
```

**e) Knowledge retrieval functions:**
```python
def get_financial_knowledge(tool_input: str, n_results: int = 3) -> str:
    results = query_knowledge_base(tool_input, n_results, metadata_filter={"domain": "general_finance"})
    return format_results_as_text(results)

def get_sector_knowledge(tool_input: Dict[str, Any]) -> str:
    query = tool_input.get("query", "")
    sector = tool_input.get("sector", None)
    n_results = tool_input.get("n_results", 3)
    if sector:
        sector_map = {
            "tech": "tech_sector",
            "technology": "tech_sector",
            "finance": "finance_sector",
            "financial": "finance_sector",
            "healthcare": "healthcare_sector",
            "health": "healthcare_sector"
        }
        domain = sector_map.get(sector.lower(), None)
        if domain:
            results = query_knowledge_base(query, n_results, metadata_filter={"domain": domain})
            return format_results_as_text(results)
        else:
            return f"Unknown sector: {sector}"
    else:
        # Query all sector domains
        domains = ["tech_sector", "finance_sector", "healthcare_sector"]
        flat_results = []
        for domain in domains:
            collection_results = query_knowledge_base(query, n_results, metadata_filter={"domain": domain})
            for result in collection_results:
                result['collection'] = domain
                flat_results.append(result)
        if flat_results and flat_results[0].get('distance') is not None:
            flat_results.sort(key=lambda x: x.get('distance', float('inf')))
        return format_results_as_text(flat_results[:n_results])

def get_investment_strategies(tool_input: str) -> str:
    results = query_knowledge_base(tool_input, n_results=3, metadata_filter={"domain": "investment_strategies"})
    return format_results_as_text(results)

def get_market_analysis_patterns(tool_input: str, n_results: int = 3) -> str:
    results = query_knowledge_base(tool_input, n_results, metadata_filter={"domain": "market_analysis"})
    return format_results_as_text(results)

def get_risk_management_guidelines(tool_input: str) -> str:
    results = query_knowledge_base(tool_input, n_results=3, metadata_filter={"domain": "risk_management"})
    return format_results_as_text(results)
```

**f) Formatting function:**
```python
def format_results_as_text(results: List[Dict[str, Any]]) -> str:
    if not results:
        return "No relevant information found."
    return "\n\n".join(f"{i+1}. {result['document']}" for i, result in enumerate(results))
```

**g) Testing block:**
```python
if __name__ == "__main__":
    print("Testing financial knowledge query:")
    print(get_financial_knowledge("What is a stock?"))
    print("\nTesting sector knowledge query:")
    print(get_sector_knowledge({"query": "cloud computing", "sector": "tech"}))
    print("\nTesting investment strategies query:")
    print(get_investment_strategies("value investing"))
    print("\nTesting market analysis patterns query:")
    print(get_market_analysis_patterns("technical analysis"))
    print("\nTesting risk management guidelines query:")
    print(get_risk_management_guidelines("diversification"))
```

- Use the same function names, arguments, and return types as in the solution.
- Use logging for info and error messages.
- Always use the ChromaDB vectorstore for retrieval, not plain text files.
- The `sector` mapping in `get_sector_knowledge` should match the solution’s logic.

Once you finish, your `kb_tools.py` should work as a drop-in replacement for the solution file, supporting all agent and team queries.

---

**General Instructions:**
- Start each file with a descriptive docstring.
- Use logging to provide setup and error information.
- Test your tools by running the file directly (add a `__main__` block with example calls).
- Refer to the solution files in `fintech_autogen/tools/` for more details and function signatures.

Once your tools are ready and tested, proceed to Step 7 to build agents and teams.

## Step 7: Scaffold Agent Files (Copy-Paste Code Templates)

Before you create the individual agent files, add the `get_global_model_client` function to the `fintech_autogen/agents/__init__.py` file. This function provides a singleton (shared instance) of the OpenAI model client, ensuring all agents use the same model client instance for efficiency and consistency.

### About `get_global_model_client`
The `get_global_model_client` function is defined in `__init__.py` so all agent modules can easily import and use it. This avoids repeated initializations and centralizes model configuration.

**Add this function to the top of `fintech_autogen/agents/__init__.py`:**
```python
from autogen_ext.models.openai import OpenAIChatCompletionClient
import os

def get_global_model_client(model_name: str = "gpt-4o-mini", api_key: str = None):
    """Singleton for global model client."""
    if not hasattr(get_global_model_client, "_client"):
        get_global_model_client._client = OpenAIChatCompletionClient(
            model=model_name,
            api_key=api_key or os.environ.get("OPENAI_API_KEY")
        )
    return get_global_model_client._client
```

You will now create the core agent files. For each agent, copy the provided code into the corresponding file under `fintech_autogen/agents/`. Each template is self-contained and ready to use—no need to refer to a separate solution project.

---

### 1. `portfolio_manager.py`
**Purpose:** Handles portfolio operations, analytics, and recommendations.

```python
"""
Portfolio Manager Agent
Handles portfolio operations, analytics, and recommendations.
"""
from typing import List, Any, Optional
from autogen_agentchat import AssistantAgent, RoundRobinGroupChat, TextMentionTermination
from fintech_autogen.tools.external_tools import setup_sql_database_toolkit
from fintech_autogen.tools.kb_tools import (
    get_investment_strategies,
    get_risk_management_guidelines,
    format_results_as_text,
)

def _get_tools_and_message():
    tools = []
    sql_tools = setup_sql_database_toolkit()
    if sql_tools:
        tools.extend(sql_tools)
    tools.append(get_investment_strategies)
    tools.append(get_risk_management_guidelines)
    default_system_message = (
        "You are a Portfolio Manager Agent responsible for managing financial portfolios.\n\n"
        "Your capabilities include:\n"
        "1. Analyzing portfolio composition and performance\n"
        "2. Managing holdings and transactions\n"
        "3. Providing investment recommendations\n"
        "4. Calculating risk metrics\n"
        "5. Generating portfolio reports\n"
        "6. Implementing investment strategies\n"
        "7. Balancing portfolios based on risk-reward profiles\n\n"
        "Use your database tools to retrieve portfolio information and perform operations.\n"
        "Use your knowledge tools to provide context about investment strategies and risk management.\n"
        "Always be precise and data-driven in your analysis.\n"
        "Ensure all recommendations consider the user's risk tolerance and investment goals.\n\n"
        "When you have completed the task, include 'TERMINATE' in your final response."
    )
    return tools, default_system_message

def _build_agent(name: str = "PortfolioManager", model_name: str = "gpt-4o-mini", api_key: Optional[str] = None, system_message: Optional[str] = None):
    from fintech_autogen.agents import get_global_model_client
    tools, default_system_message = _get_tools_and_message()
    if system_message is None:
        system_message = default_system_message
    model_client = get_global_model_client(model_name, api_key)
    agent = AssistantAgent(
        name=name,
        model_client=model_client,
        tools=tools,
        system_message=system_message,
        reflect_on_tool_use=True,
    )
    return agent

def create_portfolio_manager_team():
    agent = _build_agent()
    return RoundRobinGroupChat([agent], termination=TextMentionTermination())

def get_portfolio_manager_agent():
    return _build_agent()

def _get_demo_tasks() -> List[str]:
    return [
        "Analyze the current portfolio and suggest improvements.",
        "Provide a report on portfolio risk metrics.",
        "Recommend investment strategies for a balanced portfolio.",
    ]

async def run_portfolio_manager_demo():
    agent = get_portfolio_manager_agent()
    for task in _get_demo_tasks():
        print(f"\nTask: {task}")
        response = await agent.aask(task)
        print(f"Response:\n{response}")
```

**After saving this file, open a terminal and run:**
```
python fintech_autogen/agents/portfolio_manager.py
```
This will execute the demo tasks and print the output for each.

---

### 2. `market_analyst.py`
**Purpose:** Analyzes market trends, provides sector insights, and identifies investment opportunities.

```python
"""
Market Analyst Agent
Analyzes market trends, provides sector insights, and identifies investment opportunities.
"""
from typing import List, Any, Optional
from fintech_autogen.tools.external_tools import setup_yahoo_finance
from fintech_autogen.tools.kb_tools import (
    get_sector_knowledge,
    get_market_analysis_patterns,
    format_results_as_text,
)
from autogen_agentchat import AssistantAgent, RoundRobinGroupChat, TextMentionTermination

def _get_tools_and_message():
    tools = []
    yf_tools = setup_yahoo_finance()
    if yf_tools:
        tools.extend(list(yf_tools.values()))
    def sector_knowledge_tool(query: str, sector: str = "tech", n_results: int = 3):
        return get_sector_knowledge({"query": query, "sector": sector, "n_results": n_results})
    tools.append(sector_knowledge_tool)
    tools.append(get_market_analysis_patterns)
    default_system_message = (
        "You are a Market Analyst Agent responsible for analyzing market trends, providing sector insights, and identifying investment opportunities.\n\n"
        "Your capabilities include:\n"
        "1. Analyzing sector and market trends\n"
        "2. Retrieving real-time and historical market data\n"
        "3. Identifying key market drivers\n"
        "4. Providing investment opportunity analysis\n"
        "5. Using knowledge base and external data sources\n\n"
        "Always provide data-driven, up-to-date, and actionable insights.\n"
        "When you have completed the task, include 'TERMINATE' in your final response."
    )
    return tools, default_system_message

def _build_agent(name: str = "MarketAnalyst", model_name: str = "gpt-4o-mini", api_key: Optional[str] = None, system_message: Optional[str] = None):
    from fintech_autogen.agents import get_global_model_client
    tools, default_system_message = _get_tools_and_message()
    if system_message is None:
        system_message = default_system_message
    model_client = get_global_model_client(model_name, api_key)
    agent = AssistantAgent(
        name=name,
        model_client=model_client,
        tools=tools,
        system_message=system_message,
        reflect_on_tool_use=True,
    )
    return agent

def create_market_analyst_team():
    agent = _build_agent()
    return RoundRobinGroupChat([agent], termination=TextMentionTermination())

def get_market_analyst_agent():
    return _build_agent()

def _get_demo_tasks() -> List[str]:
    return [
        "Analyze the latest trends in the technology sector.",
        "Provide a summary of current market drivers in finance.",
        "Identify investment opportunities in healthcare."
    ]

async def run_market_analyst_demo():
    agent = get_market_analyst_agent()
    for task in _get_demo_tasks():
        print(f"\nTask: {task}")
        response = await agent.aask(task)
        print(f"Response:\n{response}")
```

**After saving this file, open a terminal and run:**
```
python fintech_autogen/agents/market_analyst.py
```
This will execute the demo tasks and print the output for each.

---

### 3. `financial_educator.py`
**Purpose:** Provides financial education, retrieves knowledge from ChromaDB, and creates learning paths.

```python
"""
Financial Educator Agent
Provides financial education, retrieves knowledge from ChromaDB, and creates learning paths.
"""
from typing import List, Any, Optional
from fintech_autogen.tools.kb_tools import (
    get_investment_strategies,
    get_risk_management_guidelines,
    format_results_as_text,
)
from autogen_agentchat import AssistantAgent, RoundRobinGroupChat, TextMentionTermination

def _build_agent(name: str = "FinancialEducator", model_name: str = "gpt-4o-mini", api_key: Optional[str] = None, system_message: Optional[str] = None):
    from fintech_autogen.agents import get_global_model_client
    tools = [get_investment_strategies, get_risk_management_guidelines, format_results_as_text]
    if system_message is None:
        system_message = (
            "You are a Financial Educator Agent responsible for providing financial education, "
            "retrieving knowledge from ChromaDB, and creating personalized learning paths.\n\n"
            "Your capabilities include:\n"
            "1. Explaining financial concepts in simple terms\n"
            "2. Retrieving and summarizing knowledge from the knowledge base\n"
            "3. Creating step-by-step learning paths for users\n"
            "4. Providing investment and risk management guidelines\n"
            "5. Answering user questions with clear, educational context\n\n"
            "Always be clear, concise, and supportive in your explanations.\n"
            "When you have completed the educational task, include 'TERMINATE' in your final response."
        )
    model_client = get_global_model_client(model_name, api_key)
    agent = AssistantAgent(
        name=name,
        model_client=model_client,
        tools=tools,
        system_message=system_message,
        reflect_on_tool_use=True,
    )
    return agent

def create_financial_educator_team():
    agent = _build_agent()
    return RoundRobinGroupChat([agent], termination=TextMentionTermination())

def get_financial_educator_agent():
    return _build_agent()

def _get_demo_tasks() -> List[str]:
    return [
        "Explain the concept of compound interest.",
        "Create a learning path for understanding ETFs.",
        "Summarize risk management guidelines for new investors."
    ]

async def run_financial_educator_demo():
    agent = get_financial_educator_agent()
    for task in _get_demo_tasks():
        print(f"\nTask: {task}")
        response = await agent.aask(task)
        print(f"Response:\n{response}")
```

**After saving this file, open a terminal and run:**
```
python fintech_autogen/agents/financial_educator.py
```
This will execute the demo tasks and print the output for each.

---

### 4. `risk_assessment.py`
**Purpose:** Evaluates investment risks and suggests mitigation strategies.

```python
"""
Risk Assessment Agent
Evaluates investment risks and suggests mitigation strategies.
"""
from typing import List, Any, Optional
from fintech_autogen.tools.kb_tools import (
    get_risk_management_guidelines,
    format_results_as_text,
)
from fintech_autogen.tools.external_tools import setup_sql_database_toolkit
from autogen_agentchat import AssistantAgent, RoundRobinGroupChat, TextMentionTermination

def _get_tools_and_message():
    tools = [get_risk_management_guidelines, format_results_as_text]
    sql_tools = setup_sql_database_toolkit()
    if sql_tools:
        tools.extend(sql_tools)
    default_system_message = (
        "You are a Risk Assessment Agent responsible for evaluating investment risks, "
        "suggesting risk mitigation strategies, and balancing risk-reward profiles.\n\n"
        "Your capabilities include:\n"
        "1. Assessing portfolio and transaction risks\n"
        "2. Retrieving and summarizing risk management guidelines\n"
        "3. Suggesting risk mitigation and diversification strategies\n"
        "4. Calculating risk metrics using available data\n"
        "5. Providing actionable, data-driven advice\n\n"
        "Always be thorough, analytical, and user-focused in your responses.\n"
        "When you have completed the risk assessment, include 'TERMINATE' in your final response."
    )
    return tools, default_system_message

def _build_agent(name: str = "RiskAssessment", model_name: str = "gpt-4o-mini", api_key: Optional[str] = None, system_message: Optional[str] = None):
    from fintech_autogen.agents import get_global_model_client
    tools, default_system_message = _get_tools_and_message()
    if system_message is None:
        system_message = default_system_message
    model_client = get_global_model_client(model_name, api_key)
    agent = AssistantAgent(
        name=name,
        model_client=model_client,
        tools=tools,
        system_message=system_message,
        reflect_on_tool_use=True,
    )
    return agent

def create_risk_assessment_team():
    agent = _build_agent()
    return RoundRobinGroupChat([agent], termination=TextMentionTermination())

def get_risk_assessment_agent():
    return _build_agent()

def _get_demo_tasks() -> List[str]:
    return [
        "Assess the risk profile of the current portfolio.",
        "Suggest risk mitigation strategies for a high-volatility asset.",
        "Summarize risk management guidelines for diversification."
    ]

async def run_risk_assessment_demo():
    agent = get_risk_assessment_agent()
    for task in _get_demo_tasks():
        print(f"\nTask: {task}")
        response = await agent.aask(task)
        print(f"Response:\n{response}")
```

**After saving this file, open a terminal and run:**
```
python fintech_autogen/agents/risk_assessment.py
```
This will execute the demo tasks and print the output for each.

---

### 5. `research_agent.py`
**Purpose:** Performs web research, real-time market data, and news analysis.

```python
"""
Research Agent
Performs web research, real-time market data, and news analysis.
"""
import os
from typing import List, Any, Optional
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_ext.teams.magentic_one import MagenticOne
from autogen_agentchat import RoundRobinGroupChat, TextMentionTermination

def _build_agent(name: str = "ResearchAgent", model_name: str = "gpt-4o-mini", api_key: Optional[str] = None):
    model_client = OpenAIChatCompletionClient(
        model=model_name,
        api_key=api_key or os.environ.get("OPENAI_API_KEY")
    )
    agent = MagenticOne(client=model_client)
    return agent

def create_research_agent_team():
    agent = _build_agent()
    return RoundRobinGroupChat([agent], termination=TextMentionTermination())

def get_research_agent():
    return _build_agent()

def _get_demo_tasks() -> List[str]:
    return [
        "Find the latest news on the S&P 500.",
        "Summarize recent trends in global markets.",
        "Research the impact of interest rate changes on tech stocks."
    ]

async def run_research_agent_demo():
    agent = get_research_agent()
    for task in _get_demo_tasks():
        print(f"\nTask: {task}")
        response = await agent.aask(task)
        print(f"Response:\n{response}")
```

**After saving this file, open a terminal and run:**
```
python fintech_autogen/agents/research_agent.py
```
This will execute the demo tasks and print the output for each.

---

### Add or Update Imports in `__init__.py` in the Agents Folder

Once you have created all the agent files, add or update the import/export block in the `fintech_autogen/agents/__init__.py` file. This enables convenient imports for all your agents and helper functions.

**Here is the recommended import/export block for `fintech_autogen/agents/__init__.py`:**
```python
from fintech_autogen.agents.portfolio_manager import create_portfolio_manager_team, get_portfolio_manager_agent
from fintech_autogen.agents.market_analyst import create_market_analyst_team, get_market_analyst_agent
from fintech_autogen.agents.financial_educator import create_financial_educator_team, get_financial_educator_agent
from fintech_autogen.agents.risk_assessment import create_risk_assessment_team, get_risk_assessment_agent
from fintech_autogen.agents.research_agent import create_research_agent, get_research_agent

__all__ = [
    "create_portfolio_manager_team",
    "get_portfolio_manager_agent",
    "create_market_analyst_team",
    "get_market_analyst_agent",
    "create_financial_educator_team",
    "get_financial_educator_agent",
    "create_risk_assessment_team",
    "get_risk_assessment_agent",
    "create_research_agent",
    "get_research_agent",
    "get_global_model_client"
]
```

**Instructions:**
- After you have created all agent files, add the above import/export block to `fintech_autogen/agents/__init__.py` (below the `get_global_model_client` function).
- This enables easy imports and ensures all agents and helpers are accessible from the package.
- Make sure your `__init__.py` is present and correct before running or testing any agent files.


## Step 8: Scaffold Team Files (Copy-Paste Code Templates)

You will now create the core team files. For each team, copy the provided code into the corresponding file under `fintech_autogen/teams/`. Each template matches the actual solution logic and is ready to use.

---

### 1. `financial_education_team.py`
**Purpose:** Orchestrates financial education using a SelectorGroupChat of Financial Educator and Market Analyst agents.

```python
"""
Financial Education Flow

This module provides a function to create a financial education team as a SelectorGroupChat.
It combines Financial Educator and Market Analyst agents for sequential educational content and context.
"""
from autogen_agentchat.teams import SelectorGroupChat
from autogen_agentchat.conditions import TextMentionTermination
from fintech_autogen.agents import get_financial_educator_agent, get_market_analyst_agent

def create_financial_education_team(termination_str=None) -> SelectorGroupChat:
    """
    Create a SelectorGroupChat for financial education with educator and analyst agents.
    Optionally override the analyst's termination string.
    Returns a SelectorGroupChat instance ready for streaming.
    """
    educator_termination = "EDUCATOR_DONE"
    analyst_termination = termination_str if termination_str is not None else "ANALYST_DONE"
    educator = get_financial_educator_agent(
        system_message=(
            "You are a Financial Educator Agent responsible for providing financial education, "
            "retrieving knowledge from ChromaDB, and creating personalized learning paths.\n\n"
            "Your capabilities include:\n"
            "1. Explaining financial concepts in simple terms\n"
            "2. Retrieving and summarizing knowledge from the knowledge base\n"
            "3. Creating step-by-step learning paths for users\n"
            "4. Providing investment and risk management guidelines\n"
            "5. Answering user questions with clear, educational context\n\n"
            "Always be clear, concise, and supportive in your explanations.\n"
            f"When you have completed the educational task, include '{educator_termination}' in your final response."
        )
    )
    analyst = get_market_analyst_agent(
        system_message=(
            "You are a Market Analyst Agent responsible for analyzing market trends, providing sector insights, and identifying investment opportunities.\n\n"
            "Your capabilities include:\n"
            "1. Analyzing sector and market trends\n"
            "2. Retrieving real-time and historical market data\n"
            "3. Identifying key market drivers\n"
            "4. Providing investment opportunity analysis\n"
            "5. Using knowledge base and external data sources\n\n"
            "Always provide data-driven, up-to-date, and actionable insights.\n"
            f"When you have completed the task, include '{analyst_termination}' in your final response."
        )
    )
    return SelectorGroupChat([
        (educator, educator_termination),
        (analyst, analyst_termination)
    ], termination=TextMentionTermination())

if __name__ == "__main__":
    team = create_financial_education_team()
    print("Financial Education Team created successfully.")
```

**After saving this file, open a terminal and run:**
```
python fintech_autogen/teams/financial_education_team.py
```
This will confirm the team is created.

---

### 2. `market_research_team.py`
**Purpose:** Orchestrates market research using the MagenticOne Research Agent.

```python
"""
Market Research Flow

This module provides a function to create a market research team using the MagenticOne Research Agent.
"""
from fintech_autogen.agents import get_research_agent
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import TextMentionTermination

def create_market_research_team(termination_str: str = "TERMINATE", model_name: str = "gpt-4o-mini", api_key=None) -> RoundRobinGroupChat:
    """
    Create a RoundRobinGroupChat for market research with the MagenticOne Research Agent.
    Returns a RoundRobinGroupChat instance ready for streaming.
    """
    research_agent = get_research_agent(
        name="ResearchAgent",
        model_name=model_name,
        api_key=api_key
    )
    return research_agent

def _get_demo_tasks():
    return [
        "Find the latest news about the electric vehicle market.",
    ]

async def run_market_research_team_demo():
    from dotenv import load_dotenv
    from autogen_agentchat.ui import Console
    load_dotenv()
    print("\n" + "="*50)
    print("Market Research Team Demo")
    print("="*50)
    team = create_market_research_team()
    tasks = _get_demo_tasks()
    for i, task in enumerate(tasks, 1):
        print(f"\nTask {i}: {task}")
        # You can implement async execution here

if __name__ == "__main__":
    print("Market Research Team created successfully.")
```

**After saving this file, open a terminal and run:**
```
python fintech_autogen/teams/market_research_team.py
```
This will confirm the team is created.

---

### 3. `portfolio_optimization_team.py`
**Purpose:** Orchestrates portfolio optimization using a GraphFlow of Portfolio Manager, Market Analyst, Risk Assessment, and optionally Financial Educator agents.

```python
"""
Portfolio Optimization Flow

This module provides a function to create a portfolio optimization GraphFlow team.
It creates and runs Portfolio Manager, Market Analyst, Risk Assessment, and optionally Financial Educator agents in parallel,
then aggregates and synthesizes their outputs.
"""
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import DiGraphBuilder, GraphFlow
from autogen_agentchat.conditions import TextMentionTermination
from fintech_autogen.agents import (
    get_portfolio_manager_agent,
    get_market_analyst_agent,
    get_risk_assessment_agent,
    get_financial_educator_agent,
    get_global_model_client
)

def create_portfolio_optimization_flow(include_educator: bool = True, termination_str: str = "TERMINATE") -> GraphFlow:
    """
    Create a GraphFlow for portfolio optimization with parallel agents and aggregation.
    Returns a GraphFlow instance ready for streaming.
    """
    pm_termination = "PORTFOLIO_MANAGER_DONE"
    analyst_termination = "ANALYST_DONE"
    risk_termination = "RISK_DONE"
    educator_termination = "EDUCATOR_DONE"
    portfolio_manager = get_portfolio_manager_agent(
        system_message=(
            "You are a Portfolio Manager Agent responsible for managing financial portfolios.\n\n"
            "Your capabilities include:\n"
            "1. Analyzing portfolio composition and performance\n"
            "2. Managing holdings and transactions\n"
            "3. Providing investment recommendations\n"
            "4. Calculating risk metrics\n"
            "5. Generating portfolio reports\n"
            "6. Implementing investment strategies\n"
            "7. Balancing portfolios based on risk-reward profiles\n\n"
            "Use your database tools to retrieve portfolio information and perform operations.\n"
            "Use your knowledge tools to provide context about investment strategies and risk management.\n"
            "Always be precise and data-driven in your analysis.\n"
            "Ensure all recommendations consider the user's risk tolerance and investment goals.\n\n"
            f"When you have completed the task, include '{pm_termination}' in your final response."
        )
    )
    market_analyst = get_market_analyst_agent(
        system_message=(
            "You are a Market Analyst Agent responsible for analyzing market trends, providing sector insights, and identifying investment opportunities.\n\n"
            "Your capabilities include:\n"
            "1. Analyzing sector and market trends\n"
            "2. Retrieving real-time and historical market data\n"
            "3. Identifying key market drivers\n"
            "4. Providing investment opportunity analysis\n"
            "5. Using knowledge base and external data sources\n\n"
            "Always provide data-driven, up-to-date, and actionable insights.\n"
            f"When you have completed the task, include '{analyst_termination}' in your final response."
        )
    )
    risk_assessment = get_risk_assessment_agent(
        system_message=(
            "You are a Risk Assessment Agent responsible for evaluating investment risks, "
            "suggesting risk mitigation strategies, and balancing risk-reward profiles.\n\n"
            "Your capabilities include:\n"
            "1. Assessing portfolio and transaction risks\n"
            "2. Retrieving and summarizing risk management guidelines\n"
            "3. Suggesting risk mitigation and diversification strategies\n"
            "4. Calculating risk metrics using available data\n"
            "5. Providing actionable, data-driven advice\n\n"
            "Always be thorough, analytical, and user-focused in your responses.\n"
            f"When you have completed the risk assessment, include '{risk_termination}' in your final response."
        )
    )
    agents = [portfolio_manager, market_analyst, risk_assessment]
    terminations = [pm_termination, analyst_termination, risk_termination]
    if include_educator:
        educator = get_financial_educator_agent(
            system_message=(
                "You are a Financial Educator Agent responsible for providing financial education, "
                "retrieving knowledge from ChromaDB, and creating personalized learning paths.\n\n"
                "Your capabilities include:\n"
                "1. Explaining financial concepts in simple terms\n"
                "2. Retrieving and summarizing knowledge from the knowledge base\n"
                "3. Creating step-by-step learning paths for users\n"
                "4. Providing investment and risk management guidelines\n"
                "5. Answering user questions with clear, educational context\n\n"
                "Always be clear, concise, and supportive in your explanations.\n"
                f"When you have completed the educational task, include '{educator_termination}' in your final response."
            )
        )
        agents.append(educator)
        terminations.append(educator_termination)
    builder = DiGraphBuilder()
    for agent, term in zip(agents, terminations):
        builder.add_node(agent, termination=TextMentionTermination(term))
    return builder.build()

if __name__ == "__main__":
    team = create_portfolio_optimization_flow()
    print("Portfolio Optimization Team created successfully.")
```

**After saving this file, open a terminal and run:**
```
python fintech_autogen/teams/portfolio_optimization_team.py
```
This will confirm the team is created.

---

### 4. `master_orchestrator.py`
**Purpose:** Orchestrates all teams for end-to-end workflows using GraphFlow and intent routing.

```python
"""
Master Orchestration (GraphFlow)

This module defines the master orchestrator using GraphFlow to route queries to the appropriate team
(Financial Education, Portfolio Optimization, Market Research) based on intent.
"""
from fintech_autogen.teams.financial_education_team import create_financial_education_team
from fintech_autogen.teams.portfolio_optimization_team import create_portfolio_optimization_flow
from fintech_autogen.teams.market_research_team import create_market_research_team
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import DiGraphBuilder, GraphFlow
from autogen_agentchat.conditions import TextMentionTermination
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.ui import Console
import logging

class MasterOrchestrator:
    """GraphFlow-based orchestrator for routing queries to the correct team(s) based on use case intent."""
    def __init__(self, termination_str="TERMINATE", model_name="gpt-4o-mini"):
        self.financial_education_team = create_financial_education_team(termination_str=termination_str)
        self.portfolio_optimization_team = create_portfolio_optimization_flow(termination_str=termination_str)
        self.market_research_team = create_market_research_team(termination_str=termination_str, model_name=model_name)
        self.termination_str = termination_str
        self.model_client = OpenAIChatCompletionClient(
            model=model_name
        )
        self.logger = logging.getLogger("MasterOrchestrator")

    def _intent_router(self, query: str):
        print(f"[Orchestrator] Routing intent for query: {query}")
        q = query.lower()
        crisis_words = ["crisis", "volatility", "real-time", "web"]
        portfolio_words = ["optimize", "rebalance", "portfolio", "allocation", "returns", "risk"]
        education_words = ["educate", "learn", "explain", "teach", "education", "beginner"]
        if any(word in q for word in crisis_words) and any(word in q for word in portfolio_words):
            print("[Orchestrator] Intent detected: research+optimization")
            return "research+optimization"
        if (any(word in q for word in education_words) and any(word in q for word in portfolio_words)) or \
           ("should i invest" in q or ("invest" in q and "know" in q)):
            print("[Orchestrator] Intent detected: education+optimization")
            return "education+optimization"
        if any(word in q for word in education_words):
            print("[Orchestrator] Intent detected: education")
            return "education"
        if any(word in q for word in portfolio_words):
            print("[Orchestrator] Intent detected: optimization")
            return "optimization"
        if any(word in q for word in crisis_words) or "research" in q:
            print("[Orchestrator] Intent detected: research")
            return "research"
        print("[Orchestrator] Intent detected: optimization (default)")
        return "optimization"

    def route_query(self, query: str):
        intent = self._intent_router(query)
        if intent == "education":
            return self.financial_education_team
        elif intent == "optimization":
            return self.portfolio_optimization_team
        elif intent == "research":
            return self.market_research_team
        elif intent == "education+optimization":
            return [self.financial_education_team, self.portfolio_optimization_team]
        elif intent == "research+optimization":
            return [self.market_research_team, self.portfolio_optimization_team]
        else:
            return self.portfolio_optimization_team

if __name__ == "__main__":
    orchestrator = MasterOrchestrator()
    print("Master Orchestrator created. You can now route queries to the appropriate team.")
```

**After saving this file, open a terminal and run:**
```
python fintech_autogen/teams/master_orchestrator.py
```
This will confirm all teams are created and the orchestrator is ready.

---

**Instructions:**
- For each team, copy the code above into the corresponding file under `fintech_autogen/teams/`.
- After saving each file, run it in the terminal to confirm the team is created.
- Each file is self-contained and ready to use.
